In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Setting/activating the environment variable as java (not compulsory everytime if you are on same runtime)
import os
os.environ["JAVA_HOME"]="/lib/jvm/java-11-openjdk-amd64"

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LogFileAnalysis") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [4]:
# Example path to log file in Drive
log_path = "/content/drive/MyDrive/Colab Notebooks/Resources/http.log"

In [5]:
http_logs_df = spark.read.text(log_path)
http_logs_df.show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1331901000.000000\tCHEt7z3AzG4gyCNgci\t192.168.202.79\t50465\t192.168.229.251\t80\t1\tHEAD\t192.168.229.251\t/DEASLog02.nsf\t-\tMozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.o

In [ ]:
# These are http logs and are tab-delimited, not the usual Apache/Nginx combined logs.

In [7]:
from pyspark.sql.functions import split, col

# Split into columns by tab
cols = [
    "timestamp", "session_id", "src_ip", "src_port",
    "dst_ip", "dst_port", "connection_id", "method",
    "host", "uri", "referrer", "user_agent",
    "bytes_in", "bytes_out", "status_code", "status_msg",
    "field1", "field2", "field3", "field4",
    "field5", "field6", "field7", "field8", "field9", "field10"
]

http_logs_parsed = http_logs_df.select([split(col("value"), "\t").getItem(i).alias(cols[i]) for i in range(len(cols))])

http_logs_parsed.show(5, truncate=False)


+-----------------+------------------+--------------+--------+---------------+--------+-------------+------+---------------+--------------+--------+------------------------------------------------------------------------------+--------+---------+-----------+----------+------+------+------+-------+------+------+------+------+------+-------+
|timestamp        |session_id        |src_ip        |src_port|dst_ip         |dst_port|connection_id|method|host           |uri           |referrer|user_agent                                                                    |bytes_in|bytes_out|status_code|status_msg|field1|field2|field3|field4 |field5|field6|field7|field8|field9|field10|
+-----------------+------------------+--------------+--------+---------------+--------+-------------+------+---------------+--------------+--------+------------------------------------------------------------------------------+--------+---------+-----------+----------+------+------+------+-------+------+------+----

In [8]:
# Tunnel logs
log_path = "/content/drive/MyDrive/Colab Notebooks/Resources/tunnel.log"
tunnel_logs_df = spark.read.text(log_path)
tunnel_logs_df.show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------+
|1331903907.840000\tCAZ2pG3tEWcozhPPzi\t192.168.202.110\t60807\t192.168.27.25\t3544\tTunnel::TEREDO\tTunnel::DISCOVER |
|1331903932.830000\tCZVmMe12RoVbuz74u5\t192.168.202.110\t43126\t192.168.27.102\t3544\tTunnel::TEREDO\tTunnel::DISCOVER|
|1331903967.840000\tCAZ2pG3tEWcozhPPzi\t192.168.202.110\t60807\t192.168.27.25\t3544\tTunnel::TEREDO\tTunnel::CLOSE    |
|1331903974.460000\tCaAV5W3NRFhcBUqY8a\t192.168.202.110\t50625\t192.168.27.152\t3544\tTunnel::TEREDO\tTunnel::DISCOVER|
|1331903979.820000\tCKf4q51yEqO6jvaw66\t192.168.202.110\t44569\t192.168.27.202\t3544\tTunnel::TEREDO\tTunnel::DISCOVER|
+---------------------------------------

In [9]:
# Tunnel log structure is So the structure is:timestamp, session_id, src_ip, src_port, dst_ip, dst_port, tunnel_type, tunnel_action
from pyspark.sql.functions import split, col

tunnel_cols = [
    "timestamp", "session_id", "src_ip", "src_port",
    "dst_ip", "dst_port", "tunnel_type", "tunnel_action"
]

tunnel_logs_parsed = tunnel_logs_df.select(
    [split(col("value"), "\t").getItem(i).alias(tunnel_cols[i]) for i in range(len(tunnel_cols))]
)

tunnel_logs_parsed.show(5, truncate=False)

+-----------------+------------------+---------------+--------+--------------+--------+--------------+----------------+
|timestamp        |session_id        |src_ip         |src_port|dst_ip        |dst_port|tunnel_type   |tunnel_action   |
+-----------------+------------------+---------------+--------+--------------+--------+--------------+----------------+
|1331903907.840000|CAZ2pG3tEWcozhPPzi|192.168.202.110|60807   |192.168.27.25 |3544    |Tunnel::TEREDO|Tunnel::DISCOVER|
|1331903932.830000|CZVmMe12RoVbuz74u5|192.168.202.110|43126   |192.168.27.102|3544    |Tunnel::TEREDO|Tunnel::DISCOVER|
|1331903967.840000|CAZ2pG3tEWcozhPPzi|192.168.202.110|60807   |192.168.27.25 |3544    |Tunnel::TEREDO|Tunnel::CLOSE   |
|1331903974.460000|CaAV5W3NRFhcBUqY8a|192.168.202.110|50625   |192.168.27.152|3544    |Tunnel::TEREDO|Tunnel::DISCOVER|
|1331903979.820000|CKf4q51yEqO6jvaw66|192.168.202.110|44569   |192.168.27.202|3544    |Tunnel::TEREDO|Tunnel::DISCOVER|
+-----------------+------------------+--

In [10]:
# ssh logs
log_path = "/content/drive/MyDrive/Colab Notebooks/Resources/ssh.log"
ssh_logs_df = spark.read.text(log_path)
ssh_logs_df.show(5, truncate=False)

#These logs are having timestamp, session_id, src_ip, src_port, dst_ip, dst_port, ssh_event, status, and delimiter is tab space

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1331901011.840000\tCTHcOo3BARDOPDjYue\t192.168.202.68\t53633\t192.168.28.254\t22\tfailure\tINBOUND\tSSH-2.0-OpenSSH_5.0\tSSH-1.99-Cisco-1.25\t-\t-\t-\t-\t-                  |
|1331901030.210000\tCBHpSz2Zi3rdKbAvwd\t192.168.202.68\t35820\t192.168.23.254\t22\tfailure\tINBOUND\tSSH-2.0-OpenSSH_5.0\tSSH-1.99-Cisco-1.25\t-\t-\t-\t-\t-                  |
|1331901032.030000\tC2h6wz2S5MWTiAk6Hb\t192.168.202.68\t36254\t192.168.26.254\t22\tfailure\tINBOUND\tSSH-2.0-OpenSSH_5.0

In [11]:
from pyspark.sql.functions import split, col

ssh_cols = [
    "timestamp", "session_id", "src_ip", "src_port",
    "dst_ip", "dst_port", "ssh_event", "status"
]

ssh_logs_parsed = ssh_logs_df.select(
    [split(col("value"), "\t").getItem(i).alias(ssh_cols[i]) for i in range(len(ssh_cols))]
)

ssh_logs_parsed.show(5, truncate=False)

+-----------------+------------------+--------------+--------+--------------+--------+---------+-------+
|timestamp        |session_id        |src_ip        |src_port|dst_ip        |dst_port|ssh_event|status |
+-----------------+------------------+--------------+--------+--------------+--------+---------+-------+
|1331901011.840000|CTHcOo3BARDOPDjYue|192.168.202.68|53633   |192.168.28.254|22      |failure  |INBOUND|
|1331901030.210000|CBHpSz2Zi3rdKbAvwd|192.168.202.68|35820   |192.168.23.254|22      |failure  |INBOUND|
|1331901032.030000|C2h6wz2S5MWTiAk6Hb|192.168.202.68|36254   |192.168.26.254|22      |failure  |INBOUND|
|1331901034.340000|CeY76r1JXPbjJS8yKb|192.168.202.68|37764   |192.168.27.102|22      |failure  |INBOUND|
|1331901041.920000|CPJHML3uGn4IV2MGWi|192.168.202.68|40244   |192.168.27.101|22      |failure  |INBOUND|
+-----------------+------------------+--------------+--------+--------------+--------+---------+-------+
only showing top 5 rows

